# OpenAI tools


In [1]:
# ! pip install -U openai duckduckgo-search

## Initialize tools

We will first create some tools we can use

In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import format_tool_to_openai_function



In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")
tools = [DuckDuckGoSearchRun()]

## Using LCEL

We will first use LangChain Expression Language to create this agent

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
llm_with_tools = llm.bind(tools=[{"type": "function", "function": format_tool_to_openai_function(t)} for t in tools])

In [6]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [7]:
from langchain.agents import AgentExecutor

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(
    {
        "input": "What's the weather in LA, NYC, and SF today"
    }
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `weather in Los Angeles today`


Los Angeles Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Los Angeles area. NEWS. NWS News; Events; Pubs/Brochures/Booklets ; NWS Media Contacts; SEARCH. Search For. NWS All NOAA. ABOUT. About NWS; Organization; Strategic Plan; ... National Weather Service Los Angeles, CA 520 North Elevar Street Oxnard, CA 93030 805-988-6610 Comments? Questions? Please Contact Us. Disclaimer Information Quality Help Glossary. LOS ANGELES (KABC) -- Southern California will see mostly clear skies and mild temperatures on Monday. Los Angeles and Orange counties will see a few clouds in the morning, but they'll clear up in ... Get Los Angeles's latest local news on crime, entertainment, weather, schools, COVID, cost of living and more. Here's your go-to source for today's LA news. Downtown Los Angeles 

{'input': "What's the weather in LA, NYC, and SF today",
 'output': "Here's the weather for today in Los Angeles, New York City, and San Francisco:\n\n- Los Angeles: Mostly clear skies and mild temperatures with a few clouds in the morning. Temperatures are around 56°F.\n- New York City: Comfortable temperatures around 68°F during the day, dropping to 48°F in the evening and night.\n- San Francisco: Moderate to high chances of rain all day with temperatures around 60°F.\n\nIf you need more detailed information, feel free to ask!"}